In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import gmean
from bokeh.charts import TimeSeries, show, output_file, BoxPlot, Bar, Scatter, Histogram
from bokeh.io import output_notebook
import seaborn as sns
# import plotly.plotly as py
# import plotly.graph_objs as go
%matplotlib inline


//anaconda/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


***
## Get started with our data
We're grabbing the raw .csv file from Charley's github folder.
***

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/charleyferrari/CUNY_DATA608/master/lecture4/Data/riverkeeper_data_2013.csv")


### Have a peek
Here's the structure; looking deeper shows that the EnteroCount variable has some slop: the entries include "<" and ">" signs that we'll need to remove.

In [3]:
df.head()

,Site,Date,EnteroCount,FourDayRainTotal,SampleCount
0,Hudson above Mohawk River,10/16/2011,1733,1.5,35
1,Hudson above Mohawk River,10/21/2013,4,0.2,35
2,Hudson above Mohawk River,9/21/2013,20,0.0,35
3,Hudson above Mohawk River,8/19/2013,6,0.0,35
4,Hudson above Mohawk River,7/21/2013,31,0.0,35


### Check data types
Indeed, Date and EnteroCount are the wrong types; they need to be Date and Integer, respectively.

In [4]:
df.dtypes

Site                 object
Date                 object
EnteroCount          object
FourDayRainTotal    float64
SampleCount           int64
dtype: object

### Fix data types, stray characters
Transform 'em while you got 'em. We'll use pandas to make the Date field DateTime.

In [5]:
df['EnteroCount'].replace('<', "", regex=True, inplace=True)

In [6]:
df['EnteroCount'].replace('>', "", regex=True, inplace=True)

In [7]:
df['EnteroCount'] = df['EnteroCount'].astype(int)

In [8]:
df['Date'] = pd.to_datetime(df['Date'])

### That's better.

In [9]:
df.dtypes

Site                        object
Date                datetime64[ns]
EnteroCount                  int64
FourDayRainTotal           float64
SampleCount                  int64
dtype: object

In [10]:
df.head()

,Site,Date,EnteroCount,FourDayRainTotal,SampleCount
0,Hudson above Mohawk River,2011-10-16,1733,1.5,35
1,Hudson above Mohawk River,2013-10-21,4,0.2,35
2,Hudson above Mohawk River,2013-09-21,20,0.0,35
3,Hudson above Mohawk River,2013-08-19,6,0.0,35
4,Hudson above Mohawk River,2013-07-21,31,0.0,35


### Unique sites
There are 75 unique measuring sites.

In [11]:
len(df['Site'].unique())

75

***
## Best and Worst Places to Swim
We'll look at mean and geometric mean measurements of Enterococcus levels by site, then sort by contamination levels.
***

### A function to get some stats
We'll apply his to each variable to get max, min, count and a mean.

In [12]:
# function to calculate group stats; used with apply()
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

### Highest Geometric mean Enterococcus readings
After computing the stats, here are the worst 10 sites by overall average contamination. No surprise if you've been on the Gowanus (now a rapidly gentrifying area for Brookly hipsters). Don't swim there. There is no method in pandas to calc the geometric mean, so we have to do it in regular Python using the gmean() function from scipy. Arrrgh.

In [13]:
grouped_df = df.groupby(['Site'])

### Highest Geometric Mean Enteroccocus readings
Slightly different ranking, but all well above the 30/mL standard.

In [14]:
grouped_df['EnteroCount'].apply(gmean, axis=None).reset_index().sort_values(by='EnteroCount', ascending=0)[0:9]

,Site,EnteroCount
70,Upper Sparkill Creek,387.184599
29,Gowanus Canal,181.330620
45,Mohawk River at Waterford,169.718920
48,Newtown Creek- Metropolitan Ave. Bridge,147.728997
63,Saw Mill River,115.117277
57,Piermont Pier,106.641987
34,Hudson River above Troy Lock,105.365758
46,Newburgh Launch Ramp,102.579356
41,Kingston STP Outfall,102.225027


### Lowest Geometric Mean Enteroccocus readings
Keepin' it clean upstate. How about them Catskills?

In [15]:
grouped_df['EnteroCount'].apply(gmean, axis=None).reset_index().sort_values(by='EnteroCount', ascending=1)[0:9]

,Site,EnteroCount
10,Catskill Creek- First Bridge,0.000000
50,Norrie Point mid-channel,3.909624
59,Poughkeepsie Drinking Water Intake,4.381236
58,Port Ewen Drinking Water Intake,5.257241
68,Tivoli Landing,6.926002
42,Little Stony Point,7.887391
69,Ulster Landing Beach,8.779956
72,West Point STP Outfall,9.210815
44,Marlboro Landing,9.661643


### Plot Best and Worst
Unfortunately, Bokeh would not let me override the default alphabetical sort on the x-axis label names. I have no idea why. I tried the code in the docs: labels = cat('Site', sort=False). I do like the way it automatically 45s the long label names.

In [16]:
df_geo = grouped_df['EnteroCount'].apply(gmean, axis=None).reset_index().sort_values(by='EnteroCount', ascending=1)

In [17]:
worst = df_geo.tail(20).sort_values('EnteroCount', ascending=1)

In [18]:
best = df_geo.head(20).sort_values('EnteroCount', ascending=1)

In [19]:
output_notebook()

Loading BokehJS ...

In [20]:
p = Bar(worst, 'Site', values='EnteroCount', 
            title="Worst 20 Enterococcus readings by Site",
            color="lightblue", legend=False, xlabel="")

output_file("bar.html")

show(p)

In [21]:
p = Bar(best, 'Site', values='EnteroCount', 
            title="Best 20 Enterococcus readings by Site",
            color="lightblue", legend=False, xlabel="")

output_file("bar.html")

show(p)

INFO:bokeh.core.state:Session output file 'bar.html' already exists, will be overwritten.


### Distribution of Enterococcus Readings
Bokeh would not let me make a simple bar chart that of all the readings that wasn't a mess. So here is a histogram showing the left skew in the data. Most of the readings are below the 30 per 100Ml level.

In [22]:
p = Histogram(df_geo['EnteroCount'], bins=100, 
                  title="Distribution of Geometric Means")

output_file("histogram.html",)

show(p)

INFO:bokeh.core.state:Session output file 'histogram.html' already exists, will be overwritten.


***
## Time Span Between Measurements
***


### Indexing on Time
We need to create a new index based on the Date field. This will make stuff easier.

In [23]:
df2 = df

In [24]:
df2.index = df['Date']
df.head()

,Site,Date,EnteroCount,FourDayRainTotal,SampleCount
Date,,,,,
2011-10-16,Hudson above Mohawk River,2011-10-16,1733,1.5,35
2013-10-21,Hudson above Mohawk River,2013-10-21,4,0.2,35
2013-09-21,Hudson above Mohawk River,2013-09-21,20,0.0,35
2013-08-19,Hudson above Mohawk River,2013-08-19,6,0.0,35
2013-07-21,Hudson above Mohawk River,2013-07-21,31,0.0,35


In [25]:
df2.resample('M', how='count')

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  if __name__ == '__main__':


,Site,Date,EnteroCount,FourDayRainTotal,SampleCount
Date,,,,,
2006-09-30,18,18,18,18,18
2006-10-31,36,36,36,36,36
2006-11-30,14,14,14,14,14
2006-12-31,15,15,15,15,15
2007-01-31,0,0,0,0,0
2007-02-28,0,0,0,0,0
2007-03-31,0,0,0,0,0
2007-04-30,26,26,26,26,26
2007-05-31,29,29,29,29,29


### This lame plot ...

... is no excuse but I ran out of time trying to figure out how to do time grouping in pandas. The friggin' docs are not clear, and the examples in Bokeh and seaborn are entirely unhelpful. This is ugly and it has nothing to do with time. The overall correlation below (.14) is weak but it's clear that low water is related to high contamination in some circumstances that need to be explored.



In [26]:
p = Scatter(df2, 'FourDayRainTotal', 'EnteroCount', 
                  xlabel="",
                  title="Enterococcus counts by Site")

output_file("Scatter.html",)

show(p)

INFO:bokeh.core.state:Session output file 'Scatter.html' already exists, will be overwritten.


In [27]:
np.corrcoef(df['EnteroCount'], df['FourDayRainTotal'])[1][0]

0.14482598724767226